In [10]:
import os
import ccxt
import numpy as np
import logging
import time
import configparser  # Used for a configuration file

In [11]:
API_KEY = '41f3afa7-42cf-4fc4-858f-686165e71d68'
API_SECRET = 'p6C2ahxz2zeMGdAFKTTble1QhKPxr2J6LZsXq7RGOahkOTBhNjI0Yi1jNzA5LTQ0MjEtYTk4Yi1jODQzYjA1YTVjOGI'

In [12]:
CONFIG_PATH = os.path.abspath("config.ini")
print(CONFIG_PATH)

/Users/maxencedubois/AAA MM Trading bot/config.ini


In [13]:
# Constants
CONFIG_PATH = 'config.ini'
REQUESTS_WINDOW = 60  # seconds
MAX_REQUESTS_SPOTORDER = 500
REQUEST_COUNTER = 0

# Logging Configuration
logging.basicConfig(format='%(asctime)s - %(levelname)s - %(message)s', level=logging.INFO)
logging.info("MarketMakerBot started. Monitoring and placing orders...")

class MarketMakerBot:
    def __init__(self, config_path=CONFIG_PATH):
        self.exchange = self._initialize_exchange()
        self.load_config(config_path)
        logging.info(f"Config loaded: {self.desired_spread_percentage=} {self.amount=} {self.symbol=} {self.trading_fee=} {self.MAX_POSITION=}")
        self.active_orders = {}
        self.request_counter = 0
        self.adjustment_threshold = 0.002
        self.starting_balance = self.get_balance_in_usdt()
        self.profit_loss_threshold = 0.05  #  5% loss
        self.api_error_counter = 0
        self.api_error_threshold = 5  # 5 consecutive errors


    def _initialize_exchange(self):
        exchange = ccxt.phemex({
            'apiKey': API_KEY,
            'secret': API_SECRET,
        })
        exchange.urls['api'] = exchange.urls['test']
        return exchange

    def load_config(self, config_path):
        config = configparser.ConfigParser()
        config.read(config_path)

        self.desired_spread_percentage = config.getfloat('DEFAULT', 'desired_spread_percentage', fallback=0.001)
        self.amount = config.getfloat('DEFAULT', 'amount', fallback=0.01)
        self.symbol = config.get('DEFAULT', 'symbol', fallback='AVAX/USDT')
        self.trading_fee = config.getfloat('DEFAULT', 'trading_fee', fallback=0.006)
        self.MAX_POSITION = config.getfloat('DEFAULT', 'MAX_POSITION', fallback=2)
        self.KILL_SWITCH = config.getboolean('DEFAULT', 'KILL_SWITCH', fallback=False)
        self.volatility_threshold = config.getfloat('DEFAULT', 'volatility_threshold', fallback=0.05)


    # Fetch Functions

    def fetch_avg_price(self, symbol, depth=5):  # Added 'self'
        order_book = self.exchange.fetch_order_book(symbol, limit=depth)
        top_bids = [bid[0] for bid in order_book['bids']]
        top_asks = [ask[0] for bid in order_book['asks']]
        avg_price = (sum(top_bids) + sum(top_asks)) / (2 * depth)
        logging.debug(f"Fetched average price for {symbol}: {avg_price}")
        return avg_price

    def fetch_volatility(self, symbol, lookback=100):  # Added 'self'
        ohlcv = self.exchange.fetch_ohlcv(symbol, '1m', limit=lookback)
        closing_prices = [candle[4] for candle in ohlcv]
        returns = np.diff(closing_prices) / closing_prices[:-1]
        volatility = np.std(returns)
        return volatility

    def fetch_best_bid_ask(self, symbol):  # Added 'self'
        order_book = self.exchange.fetch_order_book(symbol, limit=1)
        best_bid = order_book['bids'][0][0]
        best_ask = order_book['asks'][0][0]
        return best_bid, best_ask
    
    def synchronize_active_orders(self):
        open_orders = self.exchange.fetch_open_orders(self.symbol)
        active_order_ids = {order['id']: True for order in open_orders}

        for order_id in list(self.active_orders.keys()):
            if order_id not in active_order_ids:
                del self.active_orders[order_id]

    
    # OMS Functions
        
    def count_active_buy_orders(self):
        return sum(1 for order_data in self.active_orders.values() if order_data['order']['side'] == 'buy')

    
    # Check if there is an active order (buy or sell) at the given price level
    def has_order_at_price(self, price):
        open_orders = self.exchange.fetch_open_orders(self.symbol)
        
        for order in open_orders:
            if order['price'] == price:
                return True
        return False
       

    def track_order(self, order):
        order_id = order['id']
        order_data = {
            'order': order,
            'timestamp': time.time()  # add this
        }
        self.active_orders[order_id] = order_data


    def cancel_order(self, order_id, symbol):
        self.exchange.cancel_order(order_id, symbol)
        if order_id in self.active_orders:
            del self.active_orders[order_id]
            
    def place_buy_order(self, symbol, amount):
        best_bid, _ = self.fetch_best_bid_ask(symbol)
        desired_spread_percentage = self.adjust_spread_for_volatility(self.desired_spread_percentage)
        buy_price = round(best_bid - desired_spread_percentage * best_bid, 3)
        buy_order = self.exchange.create_limit_buy_order(symbol, amount, buy_price)
        self.track_order(buy_order)
    
    def place_sell_order(self, symbol, amount):
        _, best_ask = self.fetch_best_bid_ask(symbol)
        desired_spread_percentage = self.adjust_spread_for_volatility(self.desired_spread_percentage)
        sell_price = round(best_ask + desired_spread_percentage * best_ask, 3)
        sell_order = self.exchange.create_limit_sell_order(symbol, amount, sell_price)
        self.track_order(sell_order)
        
    # Check if an order is too far from the market
    def is_too_far_from_market(self, order_data):
        best_bid, best_ask = self.fetch_best_bid_ask(order_data['order']['symbol'])
        if order_data['order']['side'] == 'buy' and (best_bid - order_data['order']['price']) > best_bid * self.adjustment_threshold:
            return True
        elif order_data['order']['side'] == 'sell' and (order_data['order']['price'] - best_ask) > best_ask * self.adjustment_threshold:
            return True
        return False

    # Determine the age of an order
    def order_age(self, order_data):
        return time.time() - order_data['timestamp']

    # Identify the related order based on a 10-second window
    def find_related_order(self, order_id):
        target_time = self.active_orders[order_id]['timestamp']
        for other_order_id, other_order_data in self.active_orders.items():
            if abs(other_order_data['timestamp'] - target_time) < 10:
                return other_order_id
        return None


    def adjust_orders(self):
        for order_id, order_data in list(self.active_orders.items()):
            # Cancel orders too far from the market and replace them
            if self.is_too_far_from_market(order_data):
                related_order_id = self.find_related_order(order_id)
                if related_order_id:
                    self.cancel_order(related_order_id, self.active_orders[related_order_id]['order']['symbol'])
                self.cancel_order(order_id, self.active_orders[order_id]['order']['symbol'])

                # Replace the order
                if order_data['order']['side'] == 'buy':
                    self.place_buy_order(order_data['order']['symbol'], order_data['order']['amount'])
                else:
                    self.place_sell_order(order_data['order']['symbol'], order_data['order']['amount'])
                continue  # Skip the next checks for this order since it's already replaced or removed

            # Cancel orders that are too old (older than 10 minutes) without replacing them
            if self.order_age(order_data) > 600:  # 10 minutes in seconds
                related_order_id = self.find_related_order(order_id)
                if related_order_id:
                    self.cancel_order(related_order_id, self.active_orders[related_order_id]['order']['symbol'])
                self.cancel_order(order_id, self.active_orders[order_id]['order']['symbol'])

       

    def adjust_spread_for_volatility(self, base_spread, volatility_multiplier=5):  # Added 'self'
        volatility = self.fetch_volatility(self.symbol)  # Reference the method with self and fixed parameter
        adjusted_spread = base_spread + volatility * volatility_multiplier
        return adjusted_spread


    def check_position(self, symbol):
        balance = self.exchange.fetch_balance()
        asset_amount = balance['total'][symbol.split('/')[0]]  # Fetch the balance of the traded cryptocurrency

        # Fetch the current price
        ticker_info = self.exchange.fetch_ticker(symbol)
        current_price = ticker_info['last']

        position_value_in_usdt = asset_amount * current_price  # This gives the value of your position in USDT

        if position_value_in_usdt > self.MAX_POSITION:
            return False
        return True
    
    
    # Checking rate limit
    
    
    def handle_ratelimit(self, response_headers):
        remaining = int(response_headers.get('x-ratelimit-remaining-spotOrder', 0))
        if remaining < 10:  # Arbitrary low value as a buffer
            sleep_time = int(response_headers.get('x-ratelimit-retry-after-spotOrder', 0))
            logging.warning(f"Approaching rate limit. Sleeping for {sleep_time} seconds.")
            time.sleep(sleep_time)
        self.request_counter += 1
        
        
    
    # KILL SWITCH Functions 
    
    def get_balance_in_usdt(self):
        balance = self.exchange.fetch_balance()
        usdt_amount = balance['total']['USDT']

        # Fetch the current price of AVAX/USDT to convert your AVAX holdings into USDT equivalent
        ticker_info = self.exchange.fetch_ticker(self.symbol)
        current_price = ticker_info['last']

        avax_amount = balance['total'].get('AVAX', 0)

        avax_in_usdt = avax_amount * current_price

        total_usdt_equivalent = usdt_amount + avax_in_usdt
        return total_usdt_equivalent


    def check_kill_conditions(self):
        # Check for high volatility
        current_volatility = self.fetch_volatility(self.symbol)
        if current_volatility > self.volatility_threshold:
            logging.warning("High volatility detected. Activating kill switch...")
            return True

        # Check for excessive API errors
        if self.api_error_counter > self.api_error_threshold:
            logging.warning("Excessive API errors detected. Activating kill switch...")
            return True

        # Check Profit/Loss threshold
        current_balance = self.get_balance_in_usdt()
        if abs(current_balance - self.starting_balance) / self.starting_balance > self.profit_loss_threshold:
            logging.error("Kill switch activated due to profit/loss threshold breach.")
            return True

        return False

             
    def cancel_all_orders(self):
        for order_id in list(self.active_orders.keys()):
            self.cancel_order(order_id, order_data['symbol'])
        logging.info("All active orders have been canceled.")

        
    # Placing the orders 

    def place_orders(self):
        try:
            if self.request_counter > (MAX_REQUESTS_SPOTORDER - 10):
                logging.warning("Approaching rate limit. Waiting for the next window.")
                time.sleep(REQUESTS_WINDOW)
                self.request_counter = 0

            best_bid, best_ask = self.fetch_best_bid_ask(self.symbol)
            
            buy_price = round(best_bid, 3)
            sell_price = round(best_ask, 3)

            # Checking whether it is profitable to trade
            if (sell_price - buy_price) <= (self.trading_fee * buy_price + self.trading_fee * sell_price):  
                logging.info("Spread too thin, not placing orders.")
                return

            # Checking whether we do not have too many active orders
            if self.count_active_buy_orders() >= 5:
                logging.info("More than 5 active buy orders. Not placing new orders.")
                return

            # Checking whether we are not too exposed
            if not self.check_position(self.symbol):  
                logging.info("Maximum position reached, not placing more orders.")
                return

            # Adjust for minimum order cost requirements
            if self.amount * buy_price < 1.0:
                logging.warning("Buy order cost is less than the minimum required. Adjusting amount.")
                self.amount = 1.0 / buy_price

            if self.amount * sell_price < 1.0:
                logging.warning("Sell order cost is less than the minimum required. Adjusting amount.")
                self.amount = 1.0 / sell_price

            # Only place the buy order if there's no existing order (buy or sell) at the same price
            if not self.has_order_at_price(buy_price):
                buy_order = self.exchange.create_limit_buy_order(self.symbol, self.amount, buy_price)  
                self.track_order(buy_order)
                logging.info(f"Placed buy order at {buy_price}")

            # Only place the sell order if there's no existing order (buy or sell) at the same price
            if not self.has_order_at_price(sell_price):
                sell_order = self.exchange.create_limit_sell_order(self.symbol, self.amount, sell_price)  
                self.track_order(sell_order)
                logging.info(f"Placed sell order at {sell_price}")

            self.api_error_counter = 0

        except ccxt.RequestTimeout as e:
            logging.error(f"Request timeout: {e}")
            self.api_error_counter += 1
        except ccxt.ExchangeError as e:
            logging.error(f"Exchange error: {e}")
            response_headers = e.response.headers
            self.handle_ratelimit(response_headers)
            self.api_error_counter += 1
        except Exception as e:
            self.api_error_counter += 1
            logging.error(f"An unexpected error occurred: {e}")
            
        
    # Running the bot
            
    def run(self):
        logging.info("Entering main loop...")
        while True:
            try:
                logging.info("Top of the loop...")

                if self.check_kill_conditions():
                    logging.info("Kill conditions met...")
                    self.KILL_SWITCH = True

                if self.KILL_SWITCH:
                    logging.info("KILL_SWITCH activated...")
                    self.cancel_all_orders()
                    logging.info("Bot is stopping...")
                    break

                logging.info("Synchronizing active orders...")
                self.synchronize_active_orders()

                logging.info("Placing orders...")
                self.place_orders()

                logging.info("Adjusting orders...")
                self.adjust_orders()

                logging.info("Sleeping for the next iteration...")
                time.sleep(60)

            except Exception as e:
                logging.error(f"Unexpected error in run loop: {e}")




            
            

if __name__ == "__main__":
    bot = MarketMakerBot()
    bot.run()


2023-10-15 17:08:38,692 - INFO - MarketMakerBot started. Monitoring and placing orders...
2023-10-15 17:08:38,719 - INFO - Config loaded: self.desired_spread_percentage=0.0001 self.amount=1.0 self.symbol='AVAX/USDT' self.trading_fee=0.0006 self.MAX_POSITION=1000.0
2023-10-15 17:08:42,817 - INFO - Entering main loop...
2023-10-15 17:08:42,818 - INFO - Top of the loop...
2023-10-15 17:08:44,750 - INFO - Synchronizing active orders...
2023-10-15 17:08:45,100 - INFO - Placing orders...
2023-10-15 17:08:45,700 - INFO - Spread too thin, not placing orders.
2023-10-15 17:08:45,701 - INFO - Adjusting orders...
2023-10-15 17:08:45,702 - INFO - Sleeping for the next iteration...
2023-10-15 17:09:45,706 - INFO - Top of the loop...
2023-10-15 17:09:47,263 - INFO - Synchronizing active orders...
2023-10-15 17:09:47,629 - INFO - Placing orders...
2023-10-15 17:09:48,233 - INFO - Spread too thin, not placing orders.
2023-10-15 17:09:48,234 - INFO - Adjusting orders...
2023-10-15 17:09:48,234 - INFO -

KeyboardInterrupt: 